# 📊 OSV Security Trends: Malicious Code & Vulnerabilities in Software Supply Chains  

This notebook analyzes **OSV data**, highlighting **malicious code campaigns** and **vulnerabilities** across key ecosystems like **npm, PyPI, and Maven**.  

## 🔍 Overview  
- **npm had over 19K affected packages**, making it the most impacted registry.  
- **PyPI & Maven continue to show vulnerability spikes**, especially in recent years.  
- **Malicious Code vs. Vulnerabilities:** Attackers embed **malicious dependencies** and exploit **supply chain weaknesses.**  

## 🔗 Data Source  
- **Source:** [OSV.dev](https://osv.dev) (Open Source Vulnerability)  

## 📥 Load OSV Ecosystem Summary Data  

The dataset contains a summary of **ecosystem-specific OSV vulnerabilities and malicious code incidents** across multiple years.  

We will load and preview the dataset:  

In [21]:
import pandas as pd

df = pd.read_csv("../../data/osv/processed/osv_ecosystem_summary.csv")
df.head(1)

,ecosystem,type,total_affected,most_affected_package,peak_attack_year,trend_data
0,CRAN,Vulnerability,10,readxl,2023,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0]"


## 📊 Creating a Security Trends Table  

We will use **Great Tables** to create an interactive summary of the OSV dataset, featuring:  
- **Ecosystem breakdown** (npm, PyPI, Maven, etc.)  
- **Type of threat** (🐞 Vulnerability | 💀 Malicious Code)  
- **Peak attack year**  
- **Trend data visualization** (bar charts for yearly trends)  

In [22]:
from great_tables import GT, md, nanoplot_options
import pandas as pd
import numpy as np
import ast


# Ensure `trend_data` is properly formatted as a comma-separated string
df["trend_data"] = df["trend_data"].apply(
    lambda x: ", ".join(map(str, np.array(ast.literal_eval(x))))
    if isinstance(x, str) and x.startswith("[")
    else ", ".join(map(str, x))
    if isinstance(x, (list, np.ndarray))
    else str(x)
)

df["icon"] = df["type"].replace({"Vulnerability": "bug", "Malicious Code": "skull"})

color_map = {"bug": "purple", "skull": "red"}

# Create a sorted table DataFrame with the desired columns
table_df = df[
    ["ecosystem", "icon", "total_affected", "peak_attack_year", "trend_data"]
].sort_values(["total_affected", "ecosystem"], ascending=[False, True])

# Generate the Great Table
gt_table = (
    GT(table_df)
    .tab_header(
        title=md("**OSV Security Trends**"),
        subtitle=md(
            "_Malicious Code & Vulnerability Insights Across Software Supply Chains_"
        ),
    )
    .tab_stub(rowname_col="ecosystem")
    .tab_stubhead(label="Ecosystem")
    .tab_source_note(source_note=md("*Year Trends from 2014-2024*"))
    .tab_source_note(
        source_note=md(
            "Data sourced from [OSV.dev](https://osv.dev) (Open Source Vulnerability) and analyzed for vulnerability & malicious code trends. Covers PyPI, npm, Maven, Go, RubyGems, NuGet, Packagist, Pub, CRAN, Hackage, Hex, and crates.io. Last updated: February 2025."
        )
    )
    .tab_source_note(
        source_note=md("**Legend:** Bug = Vulnerability | Skull = Malicious Code")
    )
    .tab_stubhead(label=md("*Ecosystem*"))
    .cols_label(
        ecosystem="Ecosystem",
        icon="Type",
        total_affected="Total Affected Packages",
        peak_attack_year="Peak Year",
        trend_data="Year Trend",
    )
    .fmt_nanoplot(
        "trend_data",
        plot_type="bar",
        reference_line="mean",
        options=nanoplot_options(
            data_bar_stroke_color="black",
            data_bar_stroke_width=2,
            data_bar_fill_color="darkred",
            reference_line_color="pink",
        ),
    )
    .fmt_number(columns="total_affected", sep_mark=",", decimals=0)
    .cols_align(align="left", columns=["ecosystem"])
    .cols_align(
        align="center",
        columns=["icon", "total_affected", "peak_attack_year", "trend_data"],
    )
    .fmt_icon(columns="icon", fill_color=color_map)
)

## 💾 Saving and Displaying the Table  

The interactive table will be saved as an **HTML file** and displayed in the notebook.

In [23]:
# Generate the raw HTML from the table
html_output = gt_table.as_raw_html()

# Save it to an HTML file
with open(
    "../../data/osv/processed/osv_security_trends.html", "w", encoding="utf-8"
) as f:
    f.write(html_output)

# Display the table
gt_table

GT(_tbl_data=    ecosystem   icon  total_affected  peak_attack_year  \
15        npm  skull           19230              2024   
10       PyPI    bug           11646              2022   
9        PyPI  skull            8892              2023   
4       Maven    bug            8771              2022   
7   Packagist    bug            8527              2022   
1          Go    bug            6079              2024   
16        npm    bug            4954              2020   
6       NuGet    bug            2713              2022   
14  crates.io    bug            2086              2021   
12   RubyGems    bug            1677              2022   
11   RubyGems  skull             808              2024   
5       NuGet  skull             743              2024   
3         Hex    bug              52              2022   
8         Pub    bug              25              2023   
2     Hackage    bug              22              2023   
0        CRAN    bug              10              2023   
13  crates.io  skull               2              2023   

                                           trend_data  
15           0, 0, 0, 0, 0, 0, 0, 5, 7424, 2291, 8370  
10  116, 42, 40, 147, 266, 363, 465, 1413, 1856, 8...  
9              0, 0, 0, 0, 0, 0, 0, 0, 20, 6468, 2286  
4      0, 0, 0, 3, 250, 159, 151, 393, 2811, 938, 539  
7         0, 0, 0, 0, 3, 52, 116, 376, 2087, 681, 966  
1            0, 0, 0, 0, 0, 0, 0, 375, 858, 629, 1954  
16     0, 0, 0, 50, 305, 365, 934, 597, 663, 395, 432  
6           0, 0, 0, 3, 29, 30, 15, 103, 311, 71, 106  
14        0, 0, 6, 8, 22, 40, 168, 484, 329, 222, 307  
12        0, 0, 0, 175, 106, 57, 56, 70, 235, 90, 103  
11                 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 791  
5                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 738  
3                    0, 0, 0, 0, 0, 0, 0, 0, 21, 6, 5  
8                     0, 0, 0, 0, 0, 0, 0, 0, 3, 5, 4  
2                    0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 4  
0                    0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0  
13                    0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0  , _body=<great_tables._gt_data.Body object at 0x12c66f350>, _boxhead=Boxhead([ColInfo(var='ecosystem', type=<ColInfoTypeEnum.stub: 2>, column_label='Ecosystem', column_align='left', column_width=None), ColInfo(var='icon', type=<ColInfoTypeEnum.default: 1>, column_label='Type', column_align='center', column_width=None), ColInfo(var='total_affected', type=<ColInfoTypeEnum.default: 1>, column_label='Total Affected Packages', column_align='center', column_width=None), ColInfo(var='peak_attack_year', type=<ColInfoTypeEnum.default: 1>, column_label='Peak Year', column_align='center', column_width=None), ColInfo(var='trend_data', type=<ColInfoTypeEnum.default: 1>, column_label='Year Trend', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x12c85db10>, _spanners=Spanners([]), _heading=Heading(title=Md(text='**OSV Security Trends**'), subtitle=Md(text='_Malicious Code & Vulnerability Insights Across Software Supply Chains_'), preheader=None), _stubhead=Md(text='*Ecosystem*'), _source_notes=[Md(text='*Year Trends from 2014-2024*'), Md(text='Data sourced from [OSV.dev](https://osv.dev) (Open Source Vulnerability) and analyzed for vulnerability & malicious code trends. Covers PyPI, npm, Maven, Go, RubyGems, NuGet, Packagist, Pub, CRAN, Hackage, Hex, and crates.io. Last updated: February 2025.'), Md(text='**Legend:** Bug = Vulnerability | Skull = Malicious Code')], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x12c85eb90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x12c85e150>, <great_tables._gt_data.FormatInfo object at 0x12c85c4d0>, <great_tables._gt_data.FormatInfo object at 0x12c85f550>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='tab